In [ ]:
!pip install datasets
!pip install google-generativeai
!pip install --upgrade together


In [ ]:
!pip install wikipedia
!pip install sentence_transformers

In [ ]:
#!pip install bert_score

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import re
import pandas as pd
#from datasets import load_dataset
import time
import pickle
from tqdm import tqdm

In [ ]:
from datasets import load_dataset
dataset = load_dataset("hotpot_qa", 'fullwiki')

In [ ]:
train_dataset = dataset['train']
train_df = train_dataset.to_pandas()

In [ ]:
train_df.head(1)

,id,question,answer,type,level,supporting_facts,context
0,5a7a06935542990198eaf050,Which magazine was started first Arthur's Maga...,Arthur's Magazine,comparison,medium,"{'title': ['Arthur's Magazine', 'First for Wom...",{'title': ['Radio City (Indian radio station)'...


In [ ]:
train_df.iloc[0]['supporting_facts']

{'title': array(["Arthur's Magazine", 'First for Women'], dtype=object),
 'sent_id': array([0, 0], dtype=int32)}

In [ ]:
import os
os.getcwd()

'/Users/anushka/Downloads'

In [ ]:
df=pd.read_pickle('/content/drive/MyDrive/NLP project/train_hotpot.pkl')

In [ ]:
df.head(1)

,question,answer,supporting_facts,context,W
0,Which magazine was started first Arthur's Maga...,Arthur's Magazine,"{'title': ['Arthur's Magazine', 'First for Wom...",{'title': ['Radio City (Indian radio station)'...,Arthur's Magazine > Arthur's Magazine (1844–18...


In [ ]:
prompt_examples="""Example 1
Q: Fritz von Brodowski was killed during what global war that lasted from 1939 to 1945?
A: The answer is World War II.
W: Fritz von Brodowski > Friedrich Wilhelm Konrad von Brodowski was controversially killed while in French custody during World War II.
C: Find the [war in which Fritz von Brodowski was killed -Wiki-> y1]. Fritz von Brodowski was killed in [y1 -QA(Fritz von Brodowski was killed in which war?)-> y2]. The answer is y2.

Example 2
Q: Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?
A: The answer is Jonathan Stark.
W: Henri Leconte > He won the French Open men’s doubles title in 1984. | Jonathan Stark (tennis) > During his career he won two Grand Slam doubles titles.
C: First Search [Henri Leconte Grand Slam titles  -Wiki-> y1]. Then Search [Jonathan Stark Grand Slam titles  -Wiki-> y2]. [y1+y2 -QA(Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?)-> y3]. The answer is y3.

Example 3
Q: The director of the romantic comedy “Big Stone Gap” is based out of which part of New York city?
A: The answer is Greenwich Village.
W: Big Stone Gap (film) > Big Stone Gap is a 2014 American romantic comedy film directed by Adriana Trigiani. | Adriana Trigiani > Adriana Trigiani is an Italian American film director based in Greenwich Village.
C: First search the [Big Stone Gap director -Wiki-> y1]. The name of this film’s director is [y1 -QA(Who is the director of Big Stone Gap?)-> y2]. Then determine [y2 is based out of which city -Wiki-> y3]. The city is [y3 -QA(Where is y2 based out of?)-> y4]. Answer is y4.

Example 4
Q: Are Randal Kleiser and Kyle Schickner of the same nationality?
A: The answer is yes.
W: Randal Kleiser > John Randal Kleiser (born July 20, 1946) is an American film director and producer. | Kyle Schickner > Kyle Schickner is an American film producer, writer, director, actor.
C: First find out the [Randal Kleiser nationality -Wiki-> y1]. Then figure out the [Kyle Schickner nationality -Wiki-> y2]. Then check [y1+y2 -QA(Are Randal Kleiser and Kyle Schickner of the same nationality?)-> y3]. the answer is y3.

Example 5
Q: Extras was created, written, and directed by Ricky Dene Gervais, an English comedian, actor, writer, producer, director, singer, and musician, born on which date?
A: The answer is 25 June 1961.
W: Ricky Gervais > Ricky Dene Gervais (born 25 June 1961) is an English comedian, actor, writer, producer, director, singer, and musician.
C: Search [when Ricky Dene Gervais was born -Wiki-> y1].  Then determine [y1 -QA(When Ricky Dene Gervais was born?)-> y2]. The answer is y2.

Example 6
Q: Sameera Perera is a cricketer from what island country located southeast of the Republic of India and northeast of the Maldives?
A: The answer is Sri Lanka.
W: Sameera Perera > Sameera Perera (born 20 August 1988) is a Sri Lankan cricketer.
C: Search [Sameera Perera -Wiki-> y1]. Then find [y1 -QA(Where is Sameera Perera from?)-> y2]. The answer is y2.

Example 7
Q: What screenwriter with credits for “Evolution” co-wrote a film starring Nicolas Cage and Téa Leoni?
A: The answer is David Weissman.
W: The Family Man > The Family Man is a 2000 American romantic comedy-drama film starring Nicolas Cage and Téa Leoni. | David Weissman > His film credits include “The Family Man” (2000), “Evolution” (2001), and “When in Rome” (2010).
C: First figure out the [film of Nicolas Cage and Téa Leoni -Wiki-> y1]. The name of this film is [y1 -QA(In which film did Nicolas Cage and Téa Leoni star together?)-> y2]. Then find out [who wrote y2 with credits for “Evolution” -Wiki-> y3]. The name is screenwriter is [y3 -QA(Who wrote y2 with credits for Evolution?)-> y4]. The answer is y4.

Example 8
Q: Ralph Hefferline was a psychology professor at a university that is located in what city?
A: The answer is New York City.
W: Ralph Hefferline > Ralph Franklin Hefferline was a psychology professor at Columbia University. | Columbia University > Columbia University is a private Ivy League research university in Upper Manhattan, New York City.
C: First identify the [university of psychology professor Ralph Hefferline -Wiki-> y1]. The university of this professor is [y1 -QA(Ralph Hefferlin was a psychology professor at which university?)-> y2]. Then figure out [y2 in which city -Wiki-> y3]. The name is city is [y3 -QA(y2 is in which city?)-> y4]. The answer is y4.
"""

In [ ]:
prompt_rows = df[['question', 'answer','W']].rename(columns={'question': 'Q', 'answer': 'A'}).to_dict('records')

### TOGETHER AI

In [ ]:
llama_prompt=f"""Examples to generate Chain of Abstraction( C)  given Q, A and W are:

{prompt_examples}


Now  generate C for the test examples below. Please Write "###DONE" when you have completed generating C for all of the examples below.

"""

def generate_continuation_llama(prompt_rows):
    prompt = llama_prompt

    for idx, row in enumerate(prompt_rows):
        prompt += f"Test Example {idx + 1}\n"
        prompt += f"Q: {row['Q']}\n"
        prompt += f"A: {row['A']}\n"
        prompt += f"W: {row['W']}\n"
        prompt += "C:\n\n"
    return prompt + "Respond only with C for the test examples. Use this format >> Test Example <number> C : <generated C>\nChain of Abstractions C for the above test examples are:"



In [ ]:
input_prompt=generate_continuation_llama(prompt_rows[0:5])

In [ ]:
print(input_prompt)

In [ ]:
import os
from together import Together

client = Together(api_key="<API KEY>")

response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    top_k= 30,
    max_tokens=2000,
    messages=[{
            "content": "You are an agent that converts Questions, Answers and corresponding Wikipedia Knowledge to a Chain of Abstractions. The intuition is to get to the Answer using the facts given in the Wikipedia knowledge Your task to generate Abstractions(C) given Question (Q),Answer (A) and Wikipedia Knowledge (W).","role": "system"},
             {"role": "user", "content": input_prompt}],
)
print(response.choices[0].message.content)

### DONE

Test Example 1
Q: Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?
A: Jonathan Stark
W: Jonathan Stark (tennis) > Jonathan Stark (born April 3, 1971) is a former professional tennis player from the United States. | Jonathan Stark (tennis) >  During his career he won two Grand Slam doubles titles (the 1994 French Open Men's Doubles and the 1995 Wimbledon Championships Mixed Doubles). | Henri Leconte >  He reached the men's singles final at the French Open in 1988, won the French Open men's doubles title in 1984, and helped France win the Davis Cup in 1991.
C: First search [Henri Leconte Grand Slam titles -Wiki-> y1]. Then Search [Jonathan Stark Grand Slam titles  -Wiki-> y2]. [y1+y2 -QA(Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?)-> y3]. The answer is y3.

Test Example 2
Q: Which genus of moth in the world's seventh-largest country contains only one species?
A: Crambidae
W: Indogrammodes > Indogrammodes is a 

### GEMINI

In [ ]:
gemini_prompt_old=f"""You are an agent that converts Questions, Answers and corresponding Wikipedia Knowledge to a Chain of Abstractions. The intuition is to get to the Answer using the facts given in the Wikipedia knowledge Your task to generate Abstractions(C) given Question (Q),Answer (A) and Wikipedia Knowledge (W).
Examples to generate Chain of Abstraction( C)  given Q, A and W are:

### EXAMPLES
{prompt_examples}

### TASK
Now  generate C for the test examples below. Please Write "###DONE" when you have completed generating C for all of the examples below.
Respond in this format only:
Test Example <example number> C:

"""
#PROBLEMS EXIST in above like cheat wiki searches, wrong chains, No QA after WIKI

gemini_prompt=f"""You are an agent that converts Questions, Answers, and corresponding Wikipedia Knowledge to a Chain of Abstractions. The intuition is to get to the Answer using the titles searched in the  Wikipedia knowledge.

Rules to generate C:
1.  You have 2 tools, Wiki and QA. Use **BOTH tools** to derive C. Tool Explanations are:
	Wiki Tool to get relevant articles from Wikipedia. Format: [search query -Wiki-> search query output]
	QA tool to get the focused answer from the Wikipedia articles. Format:  [input context -QA(question)-> output]
2. W is formatted to put the title before '>' and content after '>' and separate articles using '|'.
3. Use the outputs from Wiki tool as input context for QA tool. The final answer is always output of QA tool. **This is crucial**
4. You can use page titles but cannot use page content information given in W to form Wiki search Queries.
5. You cannot use answer (A) in the tool queries directly.
6. Utilize the Chain of Thought process from the Wikipedia Knowledge given. Learn what titles need to be searched in the Wiki tool and asked in the QA tool to get to the final answer.

Your task to generate Abstractions(C)  for the given Question (Q) using both the Wiki tool and the QA tool using the rules.

Examples to generate Chain of Abstraction( C)  given Q, A and W are:
### EXAMPLES
{prompt_examples}

### TASK
Now  generate C for the test examples below. Please Write '###DONE' when you have completed generating C for all of the examples below.
Respond in this format only:
Test Example <example number> C:

"""

def generate_continuation_gemini(prompt_rows):
    prompt = gemini_prompt

    for idx, row in enumerate(prompt_rows):
        prompt += f"Test Example {idx + 1}\n"
        prompt += f"Q: {row['Q']}\n"
        prompt += f"A: {row['A']}\n"
        prompt += f"W: {row['W']}\n"
        prompt += "\n"
    return prompt + "### RESPONSE\n"



In [5]:
import google.generativeai as genai

genai.configure(api_key="<API KEY>")
# Set up the model
generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 0,
  "max_output_tokens": 3000,
}


model = genai.GenerativeModel(model_name="gemini-1.0-pro-latest",#"gemini-1.5-pro-latest",
                              generation_config=generation_config)

def get_muti_c(rows):
  input_prompt=generate_continuation_gemini(rows)
  prompt_parts = [input_prompt]
  response = model.generate_content(prompt_parts)
  contents = re.findall(r'C:(.*)', response.text)

  if len(contents)==5:
    # Add each element of content to the corresponding row
    for i, row in enumerate(rows):
        row['C'] = contents[i].strip()
    return 'success',rows
  else:
    rows.append({"content":contents})
    print("FAIL")
    return 'fail',rows


In [6]:
get_muti_c(prompt_rows[0:5])

NameError: name 'prompt_rows' is not defined

In [ ]:
input_prompt=generate_continuation_gemini(prompt_rows[5:10])
print(input_prompt)

In [ ]:
len(prompt_rows)

In [ ]:
prompt_rows=prompt_rows[:5000]

In [ ]:
generated_dataset = []
failed = []

for i in tqdm(range(0, len(prompt_rows), 5)):
    batch = prompt_rows[i:i+5]

    time.sleep(4)
    try:

      status, gen_batch = get_muti_c(batch)
      if status == 'success':
          generated_dataset.extend(gen_batch)
      else:
          failed.append(gen_batch)

      # Save generated_dataset to DataFrame pickle file after every5 iterations
      if (i + 1) % 10 == 0:
          df = pd.DataFrame(generated_dataset)
          df.to_pickle(f'"/content/drive/MyDrive/NLP project/Wiki_tool/wiki_generated_dataset_2.pkl')
          print(f'Saved generated_dataset_{i+1}.pkl')

          if len(failed) > 0:
            with open('/content/drive/MyDrive/NLP project/Wiki_tool/wiki_failed_data.pkl', 'wb') as file:
              pickle.dump(failed, file)
            print('Saved failed_data.pkl')
    except:
      print('REQUEST FAIL')
      pass


# Save any remaining data
if len(generated_dataset) > 0:
    df = pd.DataFrame(generated_dataset)
    df.to_pickle('/content/drive/MyDrive/NLP project/Wiki_tool/wiki_generated_dataset_2.pkl')
    print('Saved generated_dataset_2.pkl')

if len(failed) > 0:
    with open('/content/drive/MyDrive/NLP project/Wiki_tool/wiki_failed_data.pkl', 'wb') as file:
      pickle.dump(failed, file)
    print('Saved failed_data.pkl')

### CHECK GENERATED DATA

In [2]:
#df=pd.read_pickle("/content/drive/MyDrive/NLP project/Wiki_tool/wiki_generated_dataset_2.pkl")
df=pd.read_pickle("wiki_generated_dataset_2.pkl")


In [3]:
df.head()

,Q,A,W,C
0,Which magazine was started first Arthur's Maga...,Arthur's Magazine,Arthur's Magazine > Arthur's Magazine (1844–18...,Find out [Arthur's Magazine or First for Women...
1,The Oberoi family is part of a hotel company t...,Delhi,Oberoi family > The Oberoi family is an Indian...,Find out [The Oberoi Group head office city -W...
2,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,"Allie Goertz > Allison Beth ""Allie"" Goertz (bo...",First search [who does the character Milhouse ...
3,What nationality was James Henry Miller's wife?,American,"Peggy Seeger > Margaret ""Peggy"" Seeger (born J...",First find out [wife of James Henry Miller -Wi...
4,Cadmium Chloride is slightly soluble in this c...,alcohol,Cadmium chloride > It is a hygroscopic solid ...,First find out [Cadmium Chloride is soluble in...


In [4]:
df.shape

(4880, 4)

In [71]:
import requests
import json
from bs4 import BeautifulSoup
from pprint import pprint
import wikipedia

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


def wikipedia_seach(query):
  language_code = 'en'

  number_of_results = 3
  headers = {
    # 'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
    'User-Agent': 'YOUR_APP_NAME (YOUR_EMAIL_OR_CONTACT_PAGE)'
  }

  base_url = 'https://api.wikimedia.org/core/v1/wikipedia/'
  endpoint = '/search/page'
  url = base_url + language_code + endpoint

  search_query = query
  parameters = {'q': search_query, 'limit': number_of_results}
  response = requests.get(url, headers=headers, params=parameters)
  response = json.loads(response.text)
  print(response)
  searched_titles=[]
  for page in range(len(response['pages'])):
    display_title = response['pages'][page]['title']
    searched_titles.append(display_title)
    response['pages'][page]["article_url"] = 'http://en.wikipedia.org/?curid=' + str(response['pages'][page]['id'])
    response['pages'][page]['excerpt'] = BeautifulSoup(response['pages'][page]['excerpt']).get_text()

  excerpt_text=search_query+": "+"\n".join([page['excerpt'] for page in response['pages']])
  if(len(searched_titles)!=0):
    #go to page whose title+ excerpt is most similar
    # Encode search query
    query_embedding = sentence_model.encode(search_query)
    page_texts = [page['title'] + ": " + page['excerpt'] for page in response['pages']]
    page_embeddings = sentence_model.encode(page_texts)
    # Calculate similarity scores for each page
    similarities = cosine_similarity([query_embedding], page_embeddings)[0]

    title_score_dict = dict(zip(searched_titles, similarities))

    # Get the index of the page with the highest similarity
    max_similarity_index = similarities.argmax()

    # Output the page id with the highest similarity
    max_similarity_page_id = response['pages'][max_similarity_index]['id']

    try:
      a=wikipedia.page(pageid=max_similarity_page_id)
      full_content=a.content
      return title_score_dict,excerpt_text,full_content
    except:
      return {},"",""
  else:
    return {},"",""

def QA(ques, context):
  result = question_answerer(question=ques,context=context)
  return result['answer'], result['score']

In [6]:
## CHECKING STEP

In [7]:
# first level scoring on wikipedia search
#is wikipedia top3 matching the titles in W
#if answer not found in excerpts , go to main wikipedia page contet whose title/excerpt matches highest

In [21]:
row=678
print(df.iloc[row]['Q'])
print(df.iloc[row]['W'])
print(df.iloc[row]['C'])
print(df.iloc[row]['A'])

What was the franchise, in which Anton Šťastný played 9 seasons, renamed after moving in 1995?
Anton Šťastný > Anton Šťastný (born August 5, 1959) is a former Slovak professional ice hockey left winger who played nine seasons with the Quebec Nordiques of the National Hockey League from 1980 until 1989. | Quebec Nordiques >  The franchise was relocated to Denver, Colorado in May 1995 and renamed the Colorado Avalanche.
First find out the [team Anton Šťastný played for for 9 seasons -Wiki-> y1]. Then Find [the renamed franchise after y1 moved in 1995 -Wiki-> y2]. The answer is y2.
Colorado Avalanche


In [82]:
def process_list_tuple(rep):
  if(str(type(rep[0]))!="<class 'tuple'>"): #if wikipedia article comes as in query, return empty string as it is anyways invalid
    return ""
  #for QA result, best score string returned
  non_empty_rep = [item for item in rep if item]  # Filter out empty tuples
  if non_empty_rep:
    rep = max(non_empty_rep, key=lambda x: x[1])[0]  # Get the string with the highest score
  else:
    rep = ""  # If all tuples are empty, return an empty string
  return str(rep)


def get_variables(text):
  matches = re.findall(r'\[(.*?)\]', text)
  variables={}
  wiki_searches=[]
  errors=[]

  def replace_variable(match):
    #print(variables)
    rep = variables.get(match.group(), match.group())
    if isinstance(rep, list):
        rep=process_list_tuple(rep)
    return rep

  for instruction in matches:
    #print('at ', instruction)
    #print(variables)
    #---WIKI TOOL---
    if(instruction.find("-Wiki")!=-1):
      query,ans=instruction.split("-Wiki->") #for in query variables
      query=query.strip()
      ans=ans.strip()
      in_query_var=re.search(r'y\d+', query)
      if(in_query_var):
        #print("IN QUERY VAR",query)
        query=re.sub(r'y\d+', replace_variable, query)
        #print(query)
      try:
        title_score_dict,excerpt_text,full_content=wikipedia_seach(query)#wikipedia search
        if title_score_dict:
          #print('HERE updating varibale of wiki answer',ans.strip())
          variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content
          wiki_searches.append(title_score_dict)
        else:
          print('WIKI RESPONSE FAIL')
          errors.append(f"Error: Wikipedia search unsuccessful for query: '{query}'")
          break
      except:
        print('WIKI RESPONSE FAIL')
        errors.append(f"Error: Wikipedia search unsuccessful for query: '{query}'")
        break

    #---QA TOOL---
    if(instruction.find("-QA")!=-1):
      q,a=instruction.split("->")
      result = re.search(r'\((.*?)\)', q)
      if result:
          question = result.group(1)
      else:
          errors.append("Error in finding question")
          break

      pre=q.split(" -QA")[0]
      context_exc=""
      full_context=""
      if (pre.find("+")!=-1): #if multiple contexts need to be appended

        vars=pre.split("+")
        for i in vars:
          if(i.strip() in variables.keys()):
            if((str(type(variables[i.strip()][0]))!="<class 'tuple'>")):
              context_exc=variables[i.strip()][0] #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=variables[i.strip()][1]
            else:
              t=process_list_tuple(variables[i.strip()])
              #print('-----t',t)
              context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=t
          else:
            errors.append(f"Error: Context variable '{i.strip()}' not found")
            break
        if(context_exc==""):
          break

      else: #single context QA
        context_var=pre.strip()
        if context_var in variables.keys():
          if((str(type(variables[context_var][0]))!="<class 'tuple'>")):
            context_exc=variables[context_var][0] #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=variables[context_var][1]
          else:
            t=process_list_tuple(variables[context_var])
            context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=t
        else:
          errors.append(f"Error: Context variable '{context_var}' not found")
          break
      
      execerpt_run=QA(question, context_exc)
      if(execerpt_run[-1]<0.75 and len(full_context)>0):
        full_run=QA(question, full_context)
      else:
        full_run=()

      #print('Updating variable',a.strip())
      variables[a.strip()]=[execerpt_run,full_run]


  return variables,wiki_searches,errors


In [ ]:
##NEW
def process_list_tuple(rep):
  if(str(type(rep[0]))!="<class 'tuple'>"): #if wikipedia article comes as in query, return empty string as it is anyways invalid
    return ""
  #for QA result, best score string returned
  non_empty_rep = [item for item in rep if item]  # Filter out empty tuples
  if non_empty_rep:
    rep = max(non_empty_rep, key=lambda x: x[1])[0]  # Get the string with the highest score
  else:
    rep = ""  # If all tuples are empty, return an empty string
  return str(rep)


def get_variables(text):
  matches = re.findall(r'\[(.*?)\]', text)
  variables={}
  wiki_searches=[]
  errors=[]

  def replace_variable(match):
    #print(variables)
    rep = variables.get(match.group(), match.group())
    if isinstance(rep, list):
        rep=process_list_tuple(rep)
    return rep

  for instruction in matches:

    #print('at ', instruction)
    #print(variables)
    #---WIKI TOOL---
    if(instruction.find("-Wiki")!=-1):
      try:
        query,ans=instruction.split("-Wiki->") #for in query variables
        query=query.strip()
        ans=ans.strip()
        in_query_var=re.search(r'y\d+', query)
        if(in_query_var):
          #print("IN QUERY VAR",query)
          query=re.sub(r'y\d+', replace_variable, query)
          #print(query)
        
        title_score_dict,excerpt_text,full_content=wikipedia_seach(query)#wikipedia search

        if title_score_dict:
          #print('HERE updating varibale of wiki answer',ans.strip())
          variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content
          wiki_searches.append(title_score_dict)
        else:
          errors.append(f"Error: Wikipedia search unsuccessful for query: '{query}'")
          break
      except:
        errors.append(f"Error: Wikipedia search unsuccessful for query: '{query}'")
        break

    #---QA TOOL---
    if(instruction.find("-QA")!=-1):
      q,a=instruction.split("->")
      result = re.search(r'\((.*?)\)', q)
      if result:
          question = result.group(1)
      else:
          errors.append("Error in finding question")
          break

      pre=q.split(" -QA")[0]
      context_exc=""
      full_context=""
      if (pre.find("+")!=-1): #if multiple contexts need to be appended

        vars=pre.split("+")
        for i in vars:
          if(i.strip() in variables.keys()):
            if((str(type(variables[i.strip()][0]))!="<class 'tuple'>")):
              context_exc=variables[i.strip()][0] #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=variables[i.strip()][1]
            else:
              t=process_list_tuple(variables[i.strip()])
              #print('-----t',t)
              context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=t
          else:
            errors.append(f"Error: Context variable '{i.strip()}' not found")
            break
        if(context_exc==""):
          break

      else: #single context QA
        context_var=pre.strip()
        if context_var in variables.keys():
          if((str(type(variables[context_var][0]))!="<class 'tuple'>")):
            context_exc=variables[context_var][0] #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=variables[context_var][1]
          else:
            t=process_list_tuple(variables[context_var])
            context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=t
        else:
          errors.append(f"Error: Context variable '{context_var}' not found")
          break
      
      execerpt_run=QA(question, context_exc)
      if(execerpt_run[-1]<0.75 and len(full_context)>0):
        full_run=QA(question, full_context)
      else:
        full_run=()

      #print('Updating variable',a.strip())
      variables[a.strip()]=[execerpt_run,full_run]


  return variables,wiki_searches,errors


In [9]:
# p=pd.read_pickle("processed_rows_wiki2.pkl")
# processed=p.to_dict("records")

In [40]:
processed=processed[:1179]

In [60]:
len(processed)

1179

In [92]:
row=2
text=df.iloc[row]['C']
matches = re.findall(r'\[(.*?)\]', text)
v,w,e=get_variables(text)
matches

{'httpCode': 429, 'httpReason': ''}
WIKI RESPONSE FAIL


['who does the character Milhouse get his name from -Wiki-> y1',
 'y1 -QA(Milhouse Van Houten was named after who?)-> y2',
 'Milhouse Van Houten -QA(Who is Milhouse Van Houten named after?)-> y3',
 'y2 -QA(Who is y1?)-> y4']

In [86]:
w

[]

In [30]:
processed_df.tail()

,Q,A,W,C,Variables,Wiki_Searches,Errors,Generated_answer
4875,"What actress starred in Poor Pretty Eddie, and...",Leslie Uggams,Poor Pretty Eddie > Poor Pretty Eddie is a 197...,First Search [What actress starred in Poor Pre...,{},[],[Error: Wikipedia search unsuccessful for quer...,not found
4876,Mahatma Gandhi led India to independence throu...,obstructive program,Constructive Program > Constructive Program (C...,First search [What Mahatma Gandhi led India to...,{},[],[Error: Wikipedia search unsuccessful for quer...,not found
4877,"The author of ""The Origins of Totalitarianism""...",German-born,The Origins of Totalitarianism > The Origins o...,"First Search [Who was the author of ""The Origi...",{},[],[Error: Wikipedia search unsuccessful for quer...,not found
4878,"Who was the American actor who starred in ""Man...",Bruce Campbell,Maniac Cop > Maniac Cop is a 1988 American act...,"First Search [Who starred in ""Maniac Cop"" -Wik...",{},[],[Error: Wikipedia search unsuccessful for quer...,not found
4879,The work of philosophy and social criticism kn...,society,Dialectic of Enlightenment > Dialectic of Enli...,"First search [Who was the author of ""Dialectic...",{},[],[Error: Wikipedia search unsuccessful for quer...,not found


In [ ]:
processed_df=pd.read_pickle('/content/drive/MyDrive/NLP project/Wiki_tool/processed_rows_final_wiki2.pkl')

In [ ]:
processed_df.to_csv('a.csv')

In [ ]:
processed_df[240:260]

# EXPERIMENT

In [11]:
row=95
text=df.iloc[row]['C']
matches = re.findall(r'\[(.*?)\]', text)
matches

['type of script used in autographs -Wiki-> y1',
 'y1 -QA(What script was used in autographs?)-> y2',
 'Cuneiform script invented by-Wiki-> y3',
 'y3 -QA(Who invented cuneiform script?)-> y4']

In [ ]:
def replace_variable(match):
    return variables.get(match.group(), match.group())

variables={}
wiki_searches=[]
for instruction in matches:
  #print("at isnt",instruction,)
  if(instruction.find("-Wiki")!=-1):
    query,ans=instruction.split(" -Wiki-> ")
    #for in query variables
    in_query_var=re.search(r'y\d+', query)
    if(in_query_var):
      print("FOUND INQUERY VAR")
      query=re.sub(r'y\d+', replace_variable, query)
    #wikipedia search
    title_score_dict,excerpt_text,full_content=wikipedia_seach(query)
    #replaces the answer variable with extracted content
    variables[ans.strip()]= [excerpt_text,full_content]
    wiki_searches.append(title_score_dict)

  if(instruction.find("-QA")!=-1):
    q,a=instruction.split("->")
    result = re.search(r'\((.*?)\)', text)
    if result:
        question = result.group(1)
    else:
        print("Error in finding question")
        continue

    pre=q.split(" -QA")[0]
    if (pre.find("+")!=-1):
      vars=pre.split("+")
      context_exc=""
      full_context=""
      for i in vars:
        if(i.strip() in variables.keys()):
          context_exc=context_exc+variables[i.strip()][0]
          full_context=full_context+variables[i.strip()][1]
        else:
          print('error in finding context variable')
          continue

    else:
      context_var=pre.strip()
      if context_var in variables.keys():
        context_exc=variables[context_var][0]
        full_context=variables[context_var][1]
      else:
        print('error in finding context variable')
        continue

    execerpt_run=QA(question, context_exc)
    if(execerpt_run[-1]<0.75):
      full_run=QA(question, full_context)
    else:
      full_run=()
    variables[a.strip()]=[execerpt_run,full_run]

at isnt Henri Leconte Grand Slam titles  -Wiki-> y1
at isnt Jonathan Stark Grand Slam titles  -Wiki-> y2
at isnt y1+y2 -QA(Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?)-> y3


In [ ]:
variables

{'y1': ["Henri Leconte Grand Slam titles : Noah to win the men's doubles title at the French Open in 1984. In 1985, Leconte and Noah reached a second Grand Slam doubles final at the US Open, where\nThe Grand Slam Cup was a tennis tournament held annually at the Olympiahalle in Munich, Germany from 1990 through 1999. The event was organized by the\nis the current era of professional tennis. It began in 1968 when the Grand Slam tournaments allowed professional players to compete with amateurs, ending",
  'Henri Leconte (born 4 July 1963) is a French former professional tennis player. He reached the men\'s singles final at the French Open in 1988, won the French Open men\'s doubles title in 1984, and helped France win the Davis Cup in 1991. Leconte\'s career-high singles ranking was world No. 5.\n\n\n== Biography and career ==\nLeconte first came to the tennis world\'s attention as an outstanding junior player who won the French Open junior title in 1981.  He turned professional that year

In [ ]:
wiki_searches

[{'Henri Leconte': 0.6307155,
  'Grand Slam Cup': 0.3890447,
  "Open Era tennis records – Men's singles": 0.378835},
 {'Jonathan Stark (tennis)': 0.7212882,
  'Serena Williams': 0.47569674,
  "List of Grand Slam men's doubles champions": 0.4702412}]

In [ ]:
true_wiki_pages=[i.split(">")[0].strip() for i in df.iloc[5]['W'].split("|")]
true_wiki_pages

['Jonathan Stark (tennis)', 'Jonathan Stark (tennis)', 'Henri Leconte']

In [ ]:
query="Number of legs in chicken"
t,e,f=wikipedia_seach(query)

37402
